<a href="https://colab.research.google.com/github/c2fel/ml-fs25-coding-challenge-3rdGenMCS/blob/main/pretrained_EfficientNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import models, transforms
from PIL import Image
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import os
import glob
from torch.amp import autocast, GradScaler
from datetime import datetime

In [2]:
!pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 110.1 MB/s eta 0:00:00


In [16]:
import rasterio

# Set random seed
torch.manual_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [4]:
!wget https://madm.dfki.de/files/sentinel/EuroSATallBands.zip --no-check-certificate
!unzip -q "/content/EuroSATallBands.zip"

train_dir = "/content/ds/images/remote_sensing/otherDatasets/sentinel_2/tif"

--2025-05-17 14:47:49--  https://madm.dfki.de/files/sentinel/EuroSATallBands.zip
Resolving madm.dfki.de (madm.dfki.de)... 131.246.195.183
Connecting to madm.dfki.de (madm.dfki.de)|131.246.195.183|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 2067725275 (1.9G) [application/zip]
Saving to: ‘EuroSATallBands.zip’

EuroSATallBands.zip 100%[===================>]   1.92G  19.2MB/s    in 1m 45s  

2025-05-17 14:49:35 (18.7 MB/s) - ‘EuroSATallBands.zip’ saved [2067725275/2067725275]



In [5]:
from google.colab import drive
drive.mount('/content/drive')

!unzip -q "/content/drive/MyDrive/MCS/ML-Coding-Challenge/testset.zip"

test_dir = "/content/testset"

Mounted at /content/drive


In [6]:
# Verify dataset structure
print("Training subfolders:", sorted(os.listdir(train_dir)))
train_samples = glob.glob(os.path.join(train_dir, "*", "*.tif"))
print(f"Found {len(train_samples)} training .tif files")
print("Test files (sample):", sorted(os.listdir(test_dir))[:10])
test_samples = glob.glob(os.path.join(test_dir, "*.npy"))
print(f"Found {len(test_samples)} test .npy files")

Training subfolders: ['AnnualCrop', 'Forest', 'HerbaceousVegetation', 'Highway', 'Industrial', 'Pasture', 'PermanentCrop', 'Residential', 'River', 'SeaLake']
Found 27000 training .tif files
Test files (sample): ['test_0.npy', 'test_1.npy', 'test_10.npy', 'test_100.npy', 'test_1000.npy', 'test_1001.npy', 'test_1002.npy', 'test_1003.npy', 'test_1004.npy', 'test_1005.npy']
Found 4232 test .npy files


In [10]:
# Custom Dataset for Sentinel-2 .tif files
class TIFDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = sorted([d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))])
        self.class_to_idx = {cls_name: idx for idx, cls_name in enumerate(self.classes)}
        self.images = []
        self.labels = []
        for cls_name in self.classes:
            cls_dir = os.path.join(root_dir, cls_name)
            for img_path in glob.glob(os.path.join(cls_dir, "*.tif")):
                self.images.append(img_path)
                self.labels.append(self.class_to_idx[cls_name])

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        label = self.labels[idx]
        with rasterio.open(img_path) as src:
            img = src.read([8, 4, 3]).transpose(1, 2, 0)  # B8, B4, B3
        img = img / 10000.0
        img = (img * 255).clip(0, 255).astype(np.uint8)
        img = Image.fromarray(img)
        if self.transform:
            img = self.transform(img)
        return img, label

# Custom Dataset for unlabeled .npy files
class UnlabeledNPYDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.images = sorted(glob.glob(os.path.join(root_dir, "*.npy")))

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        npy_path = self.images[idx]
        img = np.load(npy_path)
        if img.ndim == 3 and img.shape[2] > 3:
            img = img[:, :, [7, 3, 2]] / 10000.0  # B8, B4, B3
            img = (img * 255).clip(0, 255).astype(np.uint8)
        elif img.ndim == 2:
            img = np.stack([img] * 3, axis=2).astype(np.uint8)
        img = Image.fromarray(img)
        if self.transform:
            img = self.transform(img)
        return img, npy_path

# Compute dataset-specific mean/std
def compute_mean_std(dataset):
    loader = DataLoader(dataset, batch_size=64, shuffle=False, num_workers=2)
    mean = 0.0
    std = 0.0
    n_samples = 0
    for images, _ in loader:
        batch_samples = images.size(0)
        images = images.view(batch_samples, images.size(1), -1)
        mean += images.mean(2).sum(0)
        std += images.std(2).sum(0)
        n_samples += batch_samples
    mean /= n_samples
    std /= n_samples
    return mean.tolist(), std.tolist()

# Initialize dataset for mean/std computation
temp_dataset = TIFDataset(train_dir, transform=transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()]))
mean, std = compute_mean_std(temp_dataset)
print(f"Mean: {mean}, Std: {std}")

# Data transforms
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(30),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])
test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

# Load datasets
train_dataset = TIFDataset(train_dir, transform=train_transforms)
test_dataset = UnlabeledNPYDataset(test_dir, transform=test_transforms)

# Train/validation split
indices = list(range(len(train_dataset)))
train_idx, val_idx = train_test_split(indices, test_size=0.2, stratify=train_dataset.labels, random_state=42)
train_subset = Subset(train_dataset, train_idx)
val_subset = Subset(train_dataset, val_idx)
train_loader = DataLoader(train_subset, batch_size=64, shuffle=True, num_workers=2)
val_loader = DataLoader(val_subset, batch_size=64, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)

Mean: [0.22839796543121338, 0.09292010217905045, 0.102447010576725], Std: [0.05111726373434067, 0.02677196078002453, 0.018013840541243553]


In [11]:
# Model
from torchvision.models import efficientnet_b4, EfficientNet_B4_Weights
model = efficientnet_b4(weights=EfficientNet_B4_Weights.IMAGENET1K_V1)
for param in model.parameters():
    param.requires_grad = False
for param in model.features[-4:].parameters():
    param.requires_grad = True
num_features = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_features, 10)
model = model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam([
    {"params": model.features[-4:].parameters(), "lr": 1e-4},
    {"params": model.classifier.parameters(), "lr": 1e-3}
])
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", factor=0.5, patience=5)

Downloading: "https://download.pytorch.org/models/efficientnet_b4_rwightman-23ab8bcd.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b4_rwightman-23ab8bcd.pth
100%|██████████| 74.5M/74.5M [00:00<00:00, 190MB/s]


In [19]:
# Training function
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=30):
    scaler = GradScaler('cuda')  # Updated to torch.amp
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            with autocast('cuda'):  # Updated to torch.amp
                outputs = model(inputs)
                loss = criterion(outputs, labels)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            running_loss += loss.item() * inputs.size(0)
        epoch_loss = running_loss / len(train_loader.dataset)

        model.eval()
        val_preds, val_labels = [], []
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                with autocast('cuda'):  # Updated to torch.amp
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                val_preds.extend(preds.cpu().numpy())
                val_labels.extend(labels.cpu().numpy())
        val_acc = accuracy_score(val_labels, val_preds)
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Val Accuracy: {val_acc:.4f}")
        scheduler.step(epoch_loss)

# Inference function
def infer_model(model, test_loader, class_names):
    model.eval()
    predictions = []
    with torch.no_grad():
        for inputs, paths in test_loader:
            inputs = inputs.to(device)
            with autocast('cuda'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
            for path, pred in zip(paths, preds.cpu().numpy()):
                # get file name
                test_id = os.path.basename(path).replace(".npy", "")
                # get id from filename
                number = int(test_id.split("_")[-1])

                predictions.append({
                    "test_id": number,
                    "label": class_names[pred]
                })
    pred_df = pd.DataFrame(predictions)

    # Generate timestamp string
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"/content/drive/MyDrive/MCS/ML-Coding-Challenge/EfficientNet_predictions_{timestamp}.csv"

    pred_df.to_csv(filename, index=False)
    print(f"Saved: {filename}")


In [18]:
# Train and infer
print("Starting training...")
train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=30)
print("\nPerforming inference on test set...")
infer_model(model, test_loader, train_dataset.classes)

Starting training...
Epoch 1/30, Loss: 0.0562, Val Accuracy: 0.9820
Epoch 2/30, Loss: 0.0515, Val Accuracy: 0.9867
Epoch 3/30, Loss: 0.0484, Val Accuracy: 0.9839
Epoch 4/30, Loss: 0.0428, Val Accuracy: 0.9852
Epoch 5/30, Loss: 0.0394, Val Accuracy: 0.9863
Epoch 6/30, Loss: 0.0331, Val Accuracy: 0.9872
Epoch 7/30, Loss: 0.0321, Val Accuracy: 0.9848
Epoch 8/30, Loss: 0.0257, Val Accuracy: 0.9870
Epoch 9/30, Loss: 0.0279, Val Accuracy: 0.9874
Epoch 10/30, Loss: 0.0242, Val Accuracy: 0.9878
Epoch 11/30, Loss: 0.0222, Val Accuracy: 0.9883
Epoch 12/30, Loss: 0.0234, Val Accuracy: 0.9881
Epoch 13/30, Loss: 0.0213, Val Accuracy: 0.9878
Epoch 14/30, Loss: 0.0178, Val Accuracy: 0.9891
Epoch 15/30, Loss: 0.0166, Val Accuracy: 0.9874
Epoch 16/30, Loss: 0.0179, Val Accuracy: 0.9893
Epoch 17/30, Loss: 0.0144, Val Accuracy: 0.9896
Epoch 18/30, Loss: 0.0126, Val Accuracy: 0.9885
Epoch 19/30, Loss: 0.0137, Val Accuracy: 0.9893
Epoch 20/30, Loss: 0.0120, Val Accuracy: 0.9909
Epoch 21/30, Loss: 0.0106, V

TypeError: autocast.__init__() missing 1 required positional argument: 'device_type'

In [ ]:
# Save model
filename_pth = f"/content/drive/MyDrive/MCS/ML-Coding-Challenge/EfficientNet_satellite_{timestamp}.pth"
torch.save(model.state_dict(), filename_pth)
print(f"Model saved to {filename_pth}")